In [1]:
import rasterio
import numpy as np

# Path to your raster TIFF file
tif_path = r'F:\wenqu\Aviris\classification\site4a_classified_image_svm.tif'

# Open the raster file
with rasterio.open(tif_path) as src:
    # Assuming you're interested in the first band
    band1 = src.read(1)
    
    # Find the unique values (categories) in the band
    unique_values = np.unique(band1)
    
    # Print the unique values and their count
    print(f"Unique categories: {unique_values}")
    print(f"Number of unique categories: {len(unique_values)}")


Unique categories: [0 1 2 3]
Number of unique categories: 4


In [9]:
import rasterio
from rasterio.enums import Resampling

# Path to your original raster TIFF file
input_tif_path = r'F:\wenqu\Aviris\classification\site7_classified_image_svm.tif'

# Path where you want to save the new TIFF file
output_tif_path = r'F:\wenqu\Aviris\classification\veg_maks\site7_veg.tif'

# Open the original TIFF file
with rasterio.open(input_tif_path) as src:
    # Read the first band
    band1 = src.read(1)
    
    # Create a mask where pixel values are equal to 2
    mask = (band1 == 1) 
    
    # Prepare metadata for the new TIFF file
    out_meta = src.meta.copy()
    out_meta.update({
        'dtype': 'uint8',  # Change data type to uint8 if it's not already
        'count': 1,  # Number of bands
        'nodata': 0  # Set no-data value to 0
    })
    
    # Write the mask as a new TIFF file
    with rasterio.open(output_tif_path, 'w', **out_meta) as out_dataset:
        out_dataset.write(mask.astype(rasterio.uint8), 1)


In [8]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# Path to your mask raster
mask_raster_path = 'F:/wenqu/Aviris/classification/veg_maks/site7_veg.tif'
# Output path for the shapefile
output_shapefile_path = 'F:/wenqu/Aviris/classification/veg_maks/shapefile/site7_veg.shp'

# Open the raster file
with rasterio.open(mask_raster_path) as src:
    # Read the raster data
    image = src.read(1)  # Read the first band
    
    # Masking condition, assuming 1 for mask and 0 for no mask
    mask = image == 1
    
    # Extract shapes and values from the raster
    results = list(shapes(src.read(1), mask=mask, transform=src.transform))
    
    # Convert results to GeoDataFrame
    geoms = [shape(geom) for geom, _ in results]
    gdf = gpd.GeoDataFrame({'geometry': geoms})
    
    # Set the CRS (Coordinate Reference System) from the raster
    gdf.crs = src.crs
    
    # Save to shapefile
    gdf.to_file(output_shapefile_path)

print(f"Shapefile saved to {output_shapefile_path}")


Shapefile saved to F:/wenqu/Aviris/classification/veg_maks/shapefile/site7_veg.shp


In [12]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy
# Load the shapefile
shapefile_path = r'F:\wenqu\Aviris\classification\veg_maks\shapefile\site7_veg.shp'
shapes = gpd.read_file(shapefile_path)

# Load the raster file
raster_path = r'F:\wenqu\Aviris\seven_sites_trait_map\site7_sla_ymax.tif'
with rasterio.open(raster_path) as src:
    # Define a nodata value. This should be a value that's not in your dataset
    nodata_value = 0
#     nodata_value = numpy.nan  # You can change this value based on your data


    # Clip the raster with the polygon
    out_image, out_transform = mask(src, shapes.geometry, crop=True, nodata=nodata_value, filled=True)


    # Copy the metadata
    out_meta = src.meta.copy()

# Update the metadata with new shape, transform, CRS, and nodata value
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                 "nodata": nodata_value})  # Set the nodata value in the metadata

# Write the clipped raster to a new file
clipped_raster_path = r'F:\wenqu\Aviris\classification\masked_aviris_trait\site7_sla_ymax.tif'
with rasterio.open(clipped_raster_path, "w", **out_meta) as dest:
    dest.write(out_image)
